In [3]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("/Users/thomassimmons/c/d/datasets/Uncleaned_DS_jobs.csv")

In [5]:
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [7]:
df.describe()

,index,Rating,Founded
count,672.000000,672.000000,672.000000
mean,335.500000,3.518601,1635.529762
std,194.133974,1.410329,756.746640
min,0.000000,-1.000000,-1.000000
25%,167.750000,3.300000,1917.750000
50%,335.500000,3.800000,1995.000000
75%,503.250000,4.300000,2009.000000
max,671.000000,5.000000,2019.000000


In [23]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided'],
      dtype='object')

In [13]:
# Drop index column
df.drop(columns=['index'], inplace=True)

In [17]:
df['Salary Estimate']

0      $137K-$171K (Glassdoor est.)
1      $137K-$171K (Glassdoor est.)
2      $137K-$171K (Glassdoor est.)
3      $137K-$171K (Glassdoor est.)
4      $137K-$171K (Glassdoor est.)
                   ...             
667    $105K-$167K (Glassdoor est.)
668    $105K-$167K (Glassdoor est.)
669    $105K-$167K (Glassdoor est.)
670    $105K-$167K (Glassdoor est.)
671    $105K-$167K (Glassdoor est.)
Name: Salary Estimate, Length: 672, dtype: object

In [20]:
df.shape

(672, 14)

In [22]:
# Clean salary estimate column
df = df[df['Salary Estimate'] != '-1']
df['hourly'] = df['Salary Estimate'].str.lower().str.contains('per hour')
df['employer_provided'] = df['Salary Estimate'].str.lower().str.contains('employer provided')

In [27]:
# Remove extra text and extract salary range
salary = df['Salary Estimate'].str.replace(r'\(.*?\)', '', regex=True).str.replace('K', '').str.replace('$', '').str.strip()
salary_split = salary.str.split('-', expand=True)
df['min_salary'] = salary_split[0].astype(float)
df['max_salary'] = salary_split[1].astype(float)
df['avg_salary'] = (df['min_salary'] + df['max_salary'])/2

In [28]:
# Clean company name column to remove rating number
df['Company Name'] = df['Company Name'].str.replace(r'\n[\d.]+$', '', regex=True).str.strip()

In [29]:
df['Competitors'] = df['Competitors'].replace('-1', np.nan)

In [30]:
df['Competitors']

0                EmblemHealth, UnitedHealth Group, Aetna
1                                                    NaN
2                                                    NaN
3      MKS Instruments, Pfeiffer Vacuum, Agilent Tech...
4                   Commerce Signals, Cardlytics, Yodlee
                             ...                        
667                                                  NaN
668                                                  NaN
669                                                  NaN
670                                                  NaN
671                                                  NaN
Name: Competitors, Length: 672, dtype: object

In [31]:
df = pd.read_csv("/Users/thomassimmons/c/d/datasets/retail_store_sales.csv")

In [32]:
df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [34]:
# Standardize column names - remove spaces and make everything lowercase
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

In [35]:
df.columns

Index(['transaction_id', 'customer_id', 'category', 'item', 'price_per_unit',
       'quantity', 'total_spent', 'payment_method', 'location',
       'transaction_date', 'discount_applied'],
      dtype='object')

In [38]:
# Handle missing values
df.isnull().sum()

transaction_id         0
customer_id            0
category               0
item                   0
price_per_unit         0
quantity               0
total_spent            0
payment_method         0
location               0
transaction_date       0
discount_applied    3783
dtype: int64

In [39]:
df = df.dropna(subset=['price_per_unit', 'quantity', 'total_spent'])

In [40]:
df.shape

(11362, 11)

In [41]:
df['item'] = df['item'].fillna('Unknown_Item')

In [42]:
df['discount_applied'] = df['discount_applied'].fillna(False)

/var/folders/kf/nt6wlgn90dvc9wsk7ttkqs4w0000gn/T/ipykernel_67063/793432054.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['discount_applied'] = df['discount_applied'].fillna(False)


In [43]:
df['discount_applied'] = df['discount_applied'].astype(bool)

In [44]:
df.dtypes

transaction_id       object
customer_id          object
category             object
item                 object
price_per_unit      float64
quantity            float64
total_spent         float64
payment_method       object
location             object
transaction_date     object
discount_applied       bool
dtype: object

In [45]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

In [46]:
df = df.dropna(subset=['transaction_date'])